In [1]:
import pandas as pd

In [2]:
SQL_CONN = 'sqlite:////Users/alex/Downloads/data'

In [3]:
# activities              exercises_musclegroups  workout_exercises     
#                         exercises_muscles       workouts              
# equipments              muscle_groups         
# exercises               muscles

In [4]:
muscle_groups = pd.io.sql.read_sql_table('muscle_groups', SQL_CONN)

In [5]:
muscles = pd.io.sql.read_sql_table('muscles', SQL_CONN)

In [6]:
exercises = pd.io.sql.read_sql_table('exercises', SQL_CONN)

In [7]:
exercises_musclegroups = pd.io.sql.read_sql_table('exercises_musclegroups', SQL_CONN)

In [8]:
exercises_muscles = pd.io.sql.read_sql_table('exercises_muscles', SQL_CONN)

In [9]:
# pd.io.sql.read_sql_table('equipments', SQL_CONN)

In [10]:
activities = pd.io.sql.read_sql_table('activities', SQL_CONN)

In [11]:
workout_exercises = pd.io.sql.read_sql_table('workout_exercises', SQL_CONN)

In [12]:
workouts = pd.io.sql.read_sql_table('workouts', SQL_CONN)

In [13]:
muscle_groups_txt = """const MUSCLE_GROUPS = [
%s
];

export default MUSCLE_GROUPS;
""" % ',\n'.join(["  {id: %d, name: '%s'}" % (x._1, x.name) for x in muscle_groups.itertuples()])

In [14]:
with open('/Users/alex/git/aFitness2/constants/MuscleGroups.js', 'w') as fp:
    print>>fp, muscle_groups_txt,

In [15]:
muscles_txt ="""const MUSCLES = [
%s
];

export default MUSCLES;
""" % ',\n'.join(["  {id: %d, name: '%s', muscleGroup: %d}" % (x._1, x.name, x.musclegroup_id ) for x in muscles.itertuples()])

In [16]:
with open('/Users/alex/git/aFitness2/constants/Muscles.js', 'w') as fp:
    print>>fp, muscles_txt,

In [17]:
import json

In [18]:
exercises_clean = exercises.drop(labels=['guid'], axis=1)

In [19]:
exercises_muscles_clean = exercises_muscles.drop(labels=['_id'], axis=1).set_index('exercise_id')
exercises_muscles_clean = exercises_muscles_clean[exercises_muscles_clean.muscle_id != -1]

In [20]:
def get_exercises(x):
    try:
        entry = exercises_muscles_clean.ix[x]
    except KeyError:
        return []
    if isinstance(entry, pd.DataFrame):
        return entry.to_dict('records')
    else:
        return [entry.to_dict()]

exercises_clean['muscles'] = exercises_clean._id.apply(get_exercises)

In [21]:
exercises_musclegroups_clean = exercises_musclegroups.drop(labels=['_id'], axis=1).set_index('exercise_id')
exercises_musclegroups_clean = exercises_musclegroups_clean[exercises_musclegroups_clean.musclegroup_id != -1]

In [22]:
def get_mg(x):
    try:
        return [{'musclegroup_id': int(exercises_musclegroups_clean.ix[x])}]
    except KeyError:
        return []
exercises_clean['muscle_groups'] = exercises_clean._id.apply(get_mg)

In [23]:
def do_require(image):
    if image:
        return 'xxxxx%syyyyy' % image

In [24]:
exercises_txt = json.dumps(
    exercises_clean.rename(columns={'_id': 'id'}).to_dict(orient='records'), indent=2)

In [25]:
exercises_txt_fix = """const EXERCISES = %s;

export default EXERCISES;
""" % '\n'.join([a.rstrip() for a in exercises_txt.splitlines()])

In [26]:
with open('/Users/alex/git/aFitness2/constants/Exercises.js', 'w') as fp:
    print>>fp, exercises_txt_fix,

In [27]:
images = pd.DataFrame({'image': exercises['image'][~exercises.image.isnull()].unique()})

In [28]:
images_df = images.set_index('image', drop=False)
images_txt = json.dumps(
    images_df.image.apply(do_require).to_dict(), indent=2)

In [29]:
images_txt_final = """const IMAGES = %s;

export default IMAGES;
""" % '\n'.join([a.rstrip() for a in images_txt.splitlines()]).replace('"xxxxx', "require('../img/").replace('yyyyy"', "_t.jpg')")

In [30]:
with open('/Users/alex/git/aFitness2/constants/Images.js', 'w') as fp:
    print>>fp, images_txt_final,